In [2]:
import numpy as np
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
from h3.unstable import vect
from h3 import h3_to_geo, geo_to_h3
from scipy.sparse import coo_matrix
# from sklearn.preprocessing import normalize
import cartopy.crs as ccrs

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tools import from_pickle, to_pickle, pcolorhex

def h3_cell_area():
    
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    dict_h3_normalize = pd.Series(index=map_h3_to_mat.index,dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        areas = np.array([dict_coastlines[h3_orig[i1]]['area'] for i1 in range(len(h3_orig))]) *1e6 #m2
        
        
        dict_h3_normalize[map_h3_to_mat[mask_select].index] = areas#*height
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)
    lengths = np.array([dict_coastlines[h3_orig[i1]]['length_coast_D1.27'] for i1 in range(len(h3_orig))]) *1e3
    dict_h3_normalize[map_h3_to_mat[mask_select].index] = lengths

    return dict_h3_normalize    

def h3_cell_volume():
    
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    dict_h3_normalize = pd.Series(index=map_h3_to_mat.index,dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        areas = np.array([dict_coastlines[h3_orig[i1]]['area'] for i1 in range(len(h3_orig))]) *1e6 #m2
        
        if i1 == len(depth_layers)-2: #in the lowest layer, calculate difference between local bathymetry and upper depth level (otherwise this results in np.inf)
            bath_ = df_bath[h3_orig].values
            height = bath_ - depth_layers[i1]
        else:
            height = depth_layers[i1+1] - depth_layers[i1]

        # dict_h3_area[h3_orig] = areas
        # dict_h3_volume[h3_orig] = areas*height
        
        dict_h3_normalize[map_h3_to_mat[mask_select].index] = areas*height
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)
    lengths = np.array([dict_coastlines[h3_orig[i1]]['length_coast_D1.27'] for i1 in range(len(h3_orig))]) *1e3
    dict_h3_normalize[map_h3_to_mat[mask_select].index] = lengths

    return dict_h3_normalize

def h3_cell_location():
 
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    df_h3_location = pd.DataFrame(index=map_h3_to_mat.index,columns=('lon','lat'),dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        lons_ = []
        lats_ = []
        for i2 in range(len(h3_orig)):
            (lat,lon) = h3_to_geo(hex(h3_orig[i2]))
            lats_.append(lat)
            lons_.append(lon)
            
        
        df_h3_location.loc[map_h3_to_mat[mask_select].index,'lon'] = np.array(lons_)
        df_h3_location.loc[map_h3_to_mat[mask_select].index,'lat'] = np.array(lats_)
        
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)

    lons_ = []
    lats_ = []
    for i2 in range(len(h3_orig)):
        (lat,lon) = h3_to_geo(hex(h3_orig[i2]))
        lats_.append(lat)
        lons_.append(lon)
    df_h3_location.loc[map_h3_to_mat[mask_select].index,'lon'] = np.array(lons_)
    df_h3_location.loc[map_h3_to_mat[mask_select].index,'lat'] = np.array(lats_)
    
    return df_h3_location          






/nethome/kaand004/.conda/envs/py3_parcels/lib/python3.8/site-packages/h3/unstable/__init__.py:4: UserWarning: Modules under `h3.unstable` are experimental, and may change at any time.
  warnings.warn(


In [4]:
data = from_pickle('dict_results_nsteps600_Year_2019_StokesFalse_2023-02-09_16:12:28.103508.pickle')

h3_grid_res = 3
coastline_res = '50m'
depth_layers = np.array([0,5,50,500,np.inf])
data_folder = '../'
h3_index_base = np.uint64(5e17)
index_add_coast = np.uint64(1e17)     #indices for coastal cells are the ocean surface cells + index_add_coast
h3_index_upper = h3_index_base + np.uint64(1e17)
index_remove_depth = np.array([int(0),int(1e17),int(2e17),int(3e17),int(4e17)])

df_bath = from_pickle(os.path.join(data_folder,'00_data_files/bathymetry_h%i.pickle' % h3_grid_res))
filename_dict_transition = os.path.join(data_folder,'00_data_files/transitions_h3_50m_sinking_l_0.000100-0.000400-0.001600-0.006400-0.025600-0.102400_year_2015-2016-2017-2018-2019_month_01-02-03-04-05-06-07-08-09-10-11.pickle')
P = from_pickle(filename_dict_transition)
map_mat_to_h3 = P['map_mat_to_h3']
map_h3_to_mat = P['map_h3_to_mat']
dict_coastlines = from_pickle(os.path.join(data_folder,'00_data_files/coastal_properties_h3_res%i_%s_Dc.pickle' % (h3_grid_res,coastline_res)))
h3_index_coast = np.array([hex_ for hex_ in dict_coastlines.keys() if 'length_coast' in dict_coastlines[hex_].keys() ],dtype=np.uint64)
dict_h3_normalize = h3_cell_volume()
dict_h3_location = h3_cell_location()
dict_h3_area = h3_cell_area()

# n_sizes = data_mass_balance['mass_removal_coast'][420].shape[1]
n_sizes = data[2015]['mass'].shape[1]
l0 = 1.6384  #TODO: change back to 0.1024
k_arr = np.arange(0,n_sizes) #11: 0.4mm to 0.4m
l_arr = l0 / (2**k_arr)    

In [5]:
h3_orig = map_h3_to_mat.index.values.copy()
depths = np.zeros(len(h3_orig),dtype=int)

for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig[mask_select] = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        depths[mask_select] = (0 - i1)
        
mask_select = map_h3_to_mat.index > h3_index_upper
h3_orig[mask_select] = map_h3_to_mat[mask_select].index - np.uint64(1e17)
depths[mask_select] = 1

hexes = np.array([hex(h3_orig[i].astype(int)) for i in range(len(h3_orig)) ])    

In [6]:

data

{2015: {'mass': array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [5.54048796e+04, 3.85255034e+05, 1.00730395e+06, ...,
          3.13215464e+03, 1.89098802e+03, 1.15929666e+03],
         [1.17836748e+04, 8.22255511e+04, 2.17324703e+05, ...,
          1.91966468e+03, 1.41564801e+03, 1.11248769e+03],
         [8.49392074e+04, 6.05531385e+05, 1.61894747e+06, ...,
          2.34686335e+04, 1.50254799e+04, 9.65632662e+03]]),
  'num': array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00

In [7]:
types = ['mass','num','num_f']
units = ['total','concentration']

for year_ in data.keys():
    
    # type_ = types[0]
    # unit_ = units[1]
    
    for type_ in types:
        for unit_ in units:
            
            data_out = dict_h3_location.copy()
            data_out['h3_hex_ID'] = hexes
            data_out['depth_ID'] = depths

            str_output = '_'.join([str(year_),type_,unit_]) + '.csv'

            for i1,l_ in enumerate(l_arr):
                header = '_'.join([type_,unit_,'l=%2.2f[mm]'%(l_*1000)])
                if unit_ == 'total':
                    data_out[header] = data[year_][type_][:,i1]
                elif unit_ == 'concentration':
                    data_out[header] = data[year_][type_][:,i1] / dict_h3_normalize.values

            header = '_'.join([type_,unit_,'l=total'])
            if unit_ == 'total':
                data_out[header] = data[year_][type_].sum(axis=1)
            elif unit_ == 'concentration':
                data_out[header] = data[year_][type_].sum(axis=1) / dict_h3_normalize.values                       
            
            data_out.to_csv(str_output)



/tmp/ipykernel_17383/741306549.py:23: RuntimeWarning: divide by zero encountered in true_divide
  data_out[header] = data[year_][type_][:,i1] / dict_h3_normalize.values
/tmp/ipykernel_17383/741306549.py:29: RuntimeWarning: divide by zero encountered in true_divide
  data_out[header] = data[year_][type_].sum(axis=1) / dict_h3_normalize.values


In [8]:
data_out

,lon,lat,h3_hex_ID,depth_ID,num_f_concentration_l=1638.40[mm],num_f_concentration_l=819.20[mm],num_f_concentration_l=409.60[mm],num_f_concentration_l=204.80[mm],num_f_concentration_l=102.40[mm],num_f_concentration_l=51.20[mm],num_f_concentration_l=25.60[mm],num_f_concentration_l=12.80[mm],num_f_concentration_l=6.40[mm],num_f_concentration_l=3.20[mm],num_f_concentration_l=1.60[mm],num_f_concentration_l=0.80[mm],num_f_concentration_l=0.40[mm],num_f_concentration_l=0.20[mm],num_f_concentration_l=0.10[mm],num_f_concentration_l=total
0,NaN,NaN,0x0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0x1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0x2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289972169660825599,31.757227,81.957489,0x830008fffffffff,-3,1.640152e-15,4.146740e-14,3.861193e-13,1.400027e-12,2.278564e-12,3.245170e-12,7.785298e-12,1.159869e-10,5.936052e-10,5.890229e-09,2.531577e-08,4.647597e-08,1.161877e-08,8.399954e-08,4.386327e-07,6.126577e-07
289972238380302335,25.863445,82.748108,0x830009fffffffff,-3,1.199097e-16,3.036463e-15,2.832268e-14,1.034475e-13,1.723081e-13,2.460423e-13,5.765673e-13,7.307732e-12,3.500534e-11,2.176734e-09,1.106242e-08,2.367085e-08,1.149864e-09,9.100784e-09,5.575451e-08,1.029586e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690792817652006911,145.711304,44.855713,0x832eaefffffffff,1,1.671786e-04,4.313162e-03,4.184991e-02,1.578332e-01,2.972972e-01,4.733934e-01,8.850794e-01,1.704157e+00,3.412592e+00,6.496030e+00,1.292231e+01,2.862515e+01,6.800269e+01,1.566721e+02,3.651745e+02,6.448695e+02
690381737742172159,166.698853,59.892059,0x831750fffffffff,1,1.690114e-04,4.365459e-03,4.227749e-02,1.578926e-01,2.896817e-01,4.491265e-01,8.360453e-01,1.615036e+00,2.918707e+00,5.415511e+00,1.073940e+01,2.239992e+01,4.544283e+01,1.025502e+02,2.342932e+02,4.271543e+02
690505432800296959,30.387722,45.588078,0x831e58fffffffff,1,2.838590e-06,7.919064e-05,7.847930e-04,3.090128e-03,6.475677e-03,1.273828e-02,2.848024e-02,6.525500e-02,1.355931e-01,2.938687e-01,6.728506e-01,1.651080e+00,4.766381e+00,1.163309e+01,2.876285e+01,4.803262e+01
690062054736396287,91.265167,79.440659,0x830524fffffffff,1,3.361757e-05,8.730454e-04,8.586527e-03,3.425509e-02,7.313468e-02,1.432395e-01,3.210157e-01,7.074483e-01,1.544222e+00,3.267684e+00,7.063377e+00,1.684387e+01,3.790514e+01,1.036423e+02,3.019754e+02,4.735306e+02
